In [1]:
#all needed imports

import json  
import pandas as pd
import numpy as np 
from pandas.io.json import json_normalize 

#from sklearn.model_selection import train_test_split 
#from sklearn.linear_model import LinearRegression
#from sklearn import metrics

from scipy.optimize import curve_fit
import scipy

from pulp import *

In [2]:
##JSON paths
PATH_PREV_HEATS = 'data/1/previous_heats_with_properties.json'
PATH_CONSTRAINS = 'data/1/constraints.json'
PATH_PROD_SCHED = 'data/1/production_schedule.json'
PATH_SCRAP_INV = 'data/1/scrap_inventory.json'
PATH_SCRAP_ORD = 'data/1/scrap_orders.json'

In [3]:
#function to load jsons to df
def loadjsons(jsonpath, **kwargs):
    if 'table_name' in kwargs:
        with open(jsonpath) as prev_heats:
            prev_heats = json.load(prev_heats)
        prev_heats = json_normalize(data = prev_heats[kwargs["table_name"]])
    else:
        with open(jsonpath) as prev_heats:
            prev_heats = json.load(prev_heats)
        prev_heats = json_normalize(data = prev_heats)
    return prev_heats

In [4]:
#laod all jsons
prev_heats = loadjsons(PATH_PREV_HEATS)
constriants = loadjsons(PATH_CONSTRAINS, **{'table_name': "scrap_type_constraints_per_heat"})
prod_sched = loadjsons(PATH_PROD_SCHED)
scrap_inv = loadjsons(PATH_SCRAP_INV)
scrap_ord = loadjsons(PATH_SCRAP_ORD)


In [5]:
constriants.head()

,scrap_type,type,weight
0,bushlings,minimum,200
1,pig_iron,minimum,100
2,pig_iron,maximum,300


In [6]:
prod_sched.head()

,heat_seq,heat_id,steel_grade,required_weight,chemistry.cu_pct
0,101,heat-1001,ST1,1000,0.1
1,101,heat-1002,ST1,1000,0.1
2,101,heat-1003,ST1,1000,0.1
3,101,heat-1004,ST1,1000,0.1
4,101,heat-1005,ST1,1000,0.1


In [7]:
scrap_inv.head()

,scrap_type,status,weight
0,bushling,on_hand,5000
1,pig_iron,on_hand,1500
2,municipal_shred,on_hand,12000
3,skulls,on_hand,8000


In [8]:
scrap_ord.head()

,scrap_type,order_date,status,weight,price_per_ton
0,bushling,20191005,delivered,600,75
1,bushling,20191010,delivered,1000,90
2,bushling,20191014,delivered,500,95
3,bushling,20191020,delivered,1000,85
4,bushling,20191022,delivered,1500,95


In [9]:
#calculate average (mean) price per (groupby) scrap type
scrap_prices = scrap_ord.groupby('scrap_type')['price_per_ton'].mean()
scrap_prices.head()

scrap_type
bushling            88.333333
municipal_shred     41.666667
pig_iron           154.166667
skulls              71.666667
Name: price_per_ton, dtype: float64

In [10]:
X_yld = np.squeeze(np.transpose(prev_heats[['actual_recipe.bushling','actual_recipe.pig_iron','actual_recipe.municipal_shred','actual_recipe.skulls']].values))
y_yld = np.squeeze(np.transpose(prev_heats.loc[:, ['tap_weight']].values))

In [11]:
def fn(x, b, c, d, e):
    return b*x[0] + c*x[1] + d*x[2] + e*x[3]

popt_yld, pcov_yld = curve_fit(fn, X_yld, y_yld, bounds=(0, 1))
print(popt_yld)
print(pcov_yld)

#standard deviation errors
p_sigma = np.sqrt(np.diag(pcov_yld))
print(p_sigma)

[0.8630296  0.92084372 0.92571253 0.87668814]
[[ 9.34450172e-03 -2.35926274e-05 -1.08220229e-03 -8.36256810e-03]
 [-2.35926274e-05  1.98668610e-04 -1.82431159e-03  2.69224581e-03]
 [-1.08220229e-03 -1.82431159e-03  1.87503063e-02 -2.54214283e-02]
 [-8.36256810e-03  2.69224581e-03 -2.54214283e-02  4.52260411e-02]]
[0.09666696 0.01409499 0.13693176 0.21266415]


In [12]:
yield_bushling = popt_yld[0]
yield_pig_iron = popt_yld[1]
yield_municipal_shred = popt_yld[2]
yield_skulls = popt_yld[3]

In [13]:
#get input X and output y values for mutli linear regression model
X_cu = np.squeeze(np.transpose(prev_heats[['actual_recipe.bushling','actual_recipe.pig_iron','actual_recipe.municipal_shred','actual_recipe.skulls']].values))
y_cu = np.squeeze(np.transpose(prev_heats.loc[:, ['chemistry.cu_pct']].values*prev_heats.loc[:, ['tap_weight']].values))

In [14]:
def fn(x, b, c, d, e):
    return yield_bushling*b*x[0] + yield_pig_iron*c*x[1] + yield_municipal_shred*d*x[2] + yield_skulls*e*x[3]

#covaraince = average distance from regression line?
popt_cu, pcov_cu = curve_fit(fn, X_cu, y_cu, bounds=(0, 1))
print(popt_cu)
print(pcov_cu)

#standard deviation errors
p_sigma = np.sqrt(np.diag(pcov_cu))
print(p_sigma)

[2.51540639e-17 1.18978344e-20 1.40471495e-01 2.99624460e-01]
[[ 2.10824214e+13 -8.39748397e+13  1.96548638e+13  2.07539640e+13]
 [-8.39748397e+13  3.34485948e+14 -7.82886371e+13 -8.26665385e+13]
 [ 1.96548638e+13 -7.82886371e+13  1.83239706e+13  1.93486473e+13]
 [ 2.07539640e+13 -8.26665385e+13  1.93486473e+13  2.04306238e+13]]
[ 4591559.7951727  18288957.00831324  4280650.72718589  4520024.76162259]


In [15]:
cu_bushling = popt_cu[0]
cu_pig_iron = 0
cu_municipal_shred = popt_cu[1]
cu_skulls = popt_cu[2]

In [16]:
scrap_prices.head()

scrap_type
bushling            88.333333
municipal_shred     41.666667
pig_iron           154.166667
skulls              71.666667
Name: price_per_ton, dtype: float64

In [17]:
wgt_delta = pd.DataFrame({'required_weight' : []})
wgt_delta['required_weight'] = prev_heats['required_weight']
wgt_delta.head()

,required_weight
0,1000
1,1000
2,1000
3,1000
4,1000


In [18]:
wgt_delta['sum_scrap_wgt'] = prev_heats[['actual_recipe.bushling','actual_recipe.pig_iron','actual_recipe.municipal_shred','actual_recipe.skulls']].sum(axis=1)
wgt_delta.head()

,required_weight,sum_scrap_wgt
0,1000,1050
1,1000,1050
2,1000,1050
3,1000,1050
4,1000,1050


In [19]:
wgt_delta['delta'] = wgt_delta['sum_scrap_wgt'] - wgt_delta['required_weight']
wgt_delta

,required_weight,sum_scrap_wgt,delta
0,1000,1050,50
1,1000,1050,50
2,1000,1050,50
3,1000,1050,50
4,1000,1050,50
5,1200,1050,-150
6,1200,1050,-150
7,1200,1050,-150
8,1200,1050,-150
9,1200,1050,-150


In [20]:
wgt_lower_tolerance = wgt_delta['delta'].min() if wgt_delta['delta'].min() <= 0 else 0
wgt_upper_tolerance = wgt_delta['delta'].max()

In [21]:
scrap_inv.head()
bushlings_inventory = scrap_inv.loc[scrap_inv['scrap_type'] == 'bushling', 'weight'].values[0]
pig_iron_inventory = scrap_inv.loc[scrap_inv['scrap_type'] == 'pig_iron', 'weight'].values[0]
municipal_shred_inventory = scrap_inv.loc[scrap_inv['scrap_type'] == 'municipal_shred', 'weight'].values[0]
skulls_inventory = scrap_inv.loc[scrap_inv['scrap_type'] == 'skulls', 'weight'].values[0]

In [22]:
print(bushlings_inventory)

5000


In [23]:
some_tap_weight = 1050 #required weight
some_tap_weight_upper_bound = some_tap_weight + wgt_upper_tolerance
some_tap_weight_lower_bound = some_tap_weight + wgt_lower_tolerance
cu_steel_grade = 0.15
calculated_tap_weight = 990
cu_content_abs = cu_steel_grade*calculated_tap_weight

In [24]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Cost of Scrap in Steel Grade Recipes", LpMinimize)

# Create problem variables
bushlings_wgt=LpVariable("bushlings_wgt",lowBound=200,upBound=None, cat='Continuous')
pig_iron_wgt=LpVariable("pig_iron_wgt",lowBound=100, upBound=300, cat='Continuous')
municipal_shred_wgt=LpVariable("municipal_shred_wgt",lowBound=0,upBound=None, cat='Continuous')
skulls_wgt=LpVariable("skulls_wgt",lowBound=0, upBound=None, cat='Continuous')

yield_bushling = popt_yld[0]
yield_pig_iron = popt_yld[1]
yield_municipal_shred = popt_yld[2]
yield_skulls = popt_yld[3]

# The objective function is added to 'prob' first
prob += scrap_prices.bushling*bushlings_wgt + scrap_prices.pig_iron*pig_iron_wgt + scrap_prices.municipal_shred*municipal_shred_wgt + scrap_prices.skulls*skulls_wgt, "cost; to be minimized"

# The constraints are entered
prob += bushlings_wgt + pig_iron_wgt + municipal_shred_wgt + skulls_wgt >= some_tap_weight_lower_bound, "lower bound of required weight"
prob += bushlings_wgt + pig_iron_wgt + municipal_shred_wgt + skulls_wgt <= some_tap_weight_upper_bound, "upper bound of required weight"
prob += yield_bushling*cu_bushling*bushlings_wgt + yield_pig_iron*cu_pig_iron*pig_iron_wgt + yield_municipal_shred*cu_municipal_shred*municipal_shred_wgt + yield_skulls*cu_skulls*skulls_wgt <= cu_content_abs, "Cu constraint"
prob += bushlings_inventory - bushlings_wgt >= 0 , "bushlings inventory constraint"
prob += pig_iron_inventory - pig_iron_wgt >= 0, "pig_iron inventory constraint"
prob += municipal_shred_inventory - municipal_shred_wgt >= 0, "municipal_shred inventory constraint"
prob += skulls_inventory - skulls_wgt >= 0, "skulls inventory constraint"

# The problem data is written to an .lp file
prob.writeLP("Scrap_recipes.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

1

In [25]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

bushlings_wgt = 200.0
municipal_shred_wgt = 600.0
pig_iron_wgt = 100.0
skulls_wgt = 0.0
